In [1]:
import logging
import sys


date_strftime_format = "%Y-%m-%y %H:%M:%S"
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s", datefmt=date_strftime_format)

# Data

### Training data
- Source: http://mattmahoney.net/dc/text8.zip
- Stored in: `data/train.txt`

### Analogies data
- Source: https://raw.githubusercontent.com/nicholas-leonard/word2vec/refs/heads/master/questions-words.txt
- Stored in: `data/analogies.txt`

# Defining model

In [2]:
from scipy.spatial import distance

def evaluate_analogy(model, word_tuple):
    w0_embedding = model.wv[word_tuple[0]]
    w1_embedding = model.wv[word_tuple[1]]
    w2_embedding = model.wv[word_tuple[2]]
    w3_embedding = model.wv[word_tuple[3]]

    return distance.cosine(
        w1_embedding - w0_embedding,
        w3_embedding - w2_embedding,
    )


In [3]:
analogies_file_name = "../data/analogies.txt"

with open(analogies_file_name) as file:
    file_content = file.read().splitlines()

analogies = {}
last_key_added = None
for line in file_content:
    if line[0] == ":":
        last_key_added = line.replace(": ", "")
        analogies[last_key_added] = []

    else:
        analogies[last_key_added].append(
            line.lower().split(" ")
        )


In [4]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec


class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print(f"Epoch #{self.epoch} start")

    def on_epoch_end(self, model):
        print(
            f"Epoch #{self.epoch} end.\nTraining loss: {model.get_latest_training_loss()}"
        )
        self.epoch += 1

word_embedder = Word2Vec(
    compute_loss=True,
    corpus_file="../data/train.txt",
    sg=1,
    window=9,
    vector_size=100,
    epochs=50,
    min_count=0,
    callbacks=[EpochLogger()]
)

2024-12-24 12:02:19 collecting all words and their counts
2024-12-24 12:02:20 PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-12-24 12:02:24 collected 253854 word types from a corpus of 17005207 raw words and 1701 sentences
2024-12-24 12:02:24 Creating a fresh vocabulary
2024-12-24 12:02:25 Word2Vec lifecycle event {'msg': 'effective_min_count=0 retains 253854 unique words (100.00% of original 253854, drops 0)', 'datetime': '2024-12-01T12:02:25.013508', 'gensim': '4.3.3', 'python': '3.10.14 (main, Apr 15 2024, 18:28:39) [Clang 17.0.6 ]', 'platform': 'Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'prepare_vocab'}
2024-12-24 12:02:25 Word2Vec lifecycle event {'msg': 'effective_min_count=0 leaves 17005207 word corpus (100.00% of original 17005207, drops 0)', 'datetime': '2024-12-01T12:02:25.014478', 'gensim': '4.3.3', 'python': '3.10.14 (main, Apr 15 2024, 18:28:39) [Clang 17.0.6 ]', 'platform': 'Linux-5.15.153.1-microsoft-standard-WSL2-x

In [5]:
import numpy as np

report = {k: 0.0 for k in analogies.keys()}
for sub_category in report.keys():
    report[sub_category] = np.average(
        [
            evaluate_analogy(word_embedder, curr_sample) for curr_sample in analogies[sub_category]
            if all([word_embedder.wv.__contains__(sample) for sample in curr_sample])
        ]
    )

report["overall_average"] = np.average(list(report.values()))
report

{'capital-common-countries': 0.9048030996349773,
 'capital-world': 0.9850809249235741,
 'currency': 0.9378736283202299,
 'city-in-state': 0.8502208396179038,
 'family': 0.9648204714436026,
 'gram1-adjective-to-adverb': 0.991789099099487,
 'gram2-opposite': 0.8989030008516888,
 'gram3-comparative': 0.9528796570307314,
 'gram4-superlative': 0.8235985551697462,
 'gram5-present-participle': 0.9785777547995275,
 'gram6-nationality-adjective': 0.9942383889014595,
 'gram7-past-tense': 0.9957298754691126,
 'gram8-plural': 0.9741284888124389,
 'gram9-plural-verbs': 0.9153059851630441,
 'overall_average': 0.9405678406598231}